#**Weather_report_NumPy**

> INVOLVE AND EVOLVE.


> RUN ENGINES.






#Generating Data

In [ ]:
import numpy as np

# Seed for reproducibility
np.random.seed(0)

# Number of samples
n_samples = 1000

# Generate random weather data: Temperature (T), Humidity (H), Pressure (P)
temperature = np.random.normal(25, 5, n_samples)  # mean=25°C, std=5°C
humidity = np.random.normal(60, 10, n_samples)    # mean=60%, std=10%
pressure = np.random.normal(1013, 10, n_samples)  # mean=1013 hPa, std=10 hPa

# Generate random labels (0: sunny, 1: rainy)
rainy = (temperature < 22) & (humidity > 65) & (pressure < 1010)
labels = rainy.astype(int)


In [ ]:
temperature[:2]

array([33.82026173, 27.00078604])

In [ ]:
labels[30:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

#Preprocess Data

In [ ]:
# Combine into a single dataset
X = np.column_stack((temperature, humidity, pressure))
y = labels.reshape(-1, 1)

# Add a column of ones to X for the bias term
X = np.hstack((np.ones((X.shape[0], 1)), X))

# Split data into training and testing sets
split_ratio = 0.8
split_index = int(n_samples * split_ratio)

X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]


In [ ]:
X

array([[1.00000000e+00, 3.38202617e+01, 6.55596268e+01, 9.97670789e+02],
       [1.00000000e+00, 2.70007860e+01, 6.89247389e+01, 9.95880298e+02],
       [1.00000000e+00, 2.98936899e+01, 5.57768518e+01, 1.01346135e+03],
       ...,
       [1.00000000e+00, 2.54709615e+01, 6.15843385e+01, 1.01107596e+03],
       [1.00000000e+00, 1.92619453e+01, 4.85809858e+01, 1.00087484e+03],
       [1.00000000e+00, 2.32094296e+01, 4.68902963e+01, 1.01219401e+03]])

In [ ]:
y[12:45]

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0]])

#Initialize Parameters

In [ ]:
# Initialize weights
weights = np.zeros((X.shape[1], 1))

In [ ]:
weights

array([[0.],
       [0.],
       [0.],
       [0.]])

#Define Logistic Regression Functions



1. **sigmoid(z)**

**Purpose**: To apply the sigmoid function, which is used to map predictions to probabilities in logistic regression.

**Explanation**:
The sigmoid function takes any real-valued number and maps it to a value between 0 and 1. This is useful for binary classification, where we want to predict the probability that a given input belongs to a particular class.

**Usage**:

z is typically the dot product of the feature matrix
𝑋
X and the weight vector
𝑤
w.
Returns the sigmoid-transformed value of
𝑧
z.

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


2. **predict(X, weights)**
Purpose: To compute the predicted probabilities for the input features
𝑋
X using the current weights.

Explanation:
This function calculates the dot product of the feature matrix
𝑋
X and the weight vector
𝑤
w, and then applies the sigmoid function to the result to obtain the probabilities.

**Usage**:

X is the matrix of input features (with an added bias term).
weights is the vector of model parameters.
Returns the predicted probabilities for each sample in
𝑋
X.

In [ ]:
def predict(X, weights):
  return sigmoid(np.dot(X, weights))

3. compute_cost(X, y, weights)

Purpose: To compute the cost (or loss) of the logistic regression model.

Explanation:
The cost function measures how well the model's predictions match the actual data. For logistic regression, the cost function is derived from the likelihood of the observed data and is given by the binary cross-entropy (log-loss) function.

Mathematical Formulation:

𝐽
(
𝑤
)
=
−
1
𝑚
∑
𝑖
=
1
𝑚
[
𝑦
(
𝑖
)
log
⁡
(
ℎ
𝑤
(
𝑥
(
𝑖
)
)
)
+
(
1
−
𝑦
(
𝑖
)
)
log
⁡
(
1
−
ℎ
𝑤
(
𝑥
(
𝑖
)
)
)
]
J(w)=−
m
1
​
  
i=1
∑
m
​
 [y
(i)
 log(h
w
​
 (x
(i)
 ))+(1−y
(i)
 )log(1−h
w
​
 (x
(i)
 ))]
where
ℎ
𝑤
(
𝑥
)
=
𝜎
(
𝑤
𝑇
𝑥
)
h
w
​
 (x)=σ(w
T
 x) is the hypothesis.

 **Usage**:

X is the matrix of input features.
y is the vector of true labels.
weights is the vector of model parameters.
Returns the computed cost (scalar value).

In [ ]:
def compute_cost(X, y, weights):
    m = len(y)
    h = predict(X, weights)
    cost = -1/m * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))
    return cost

4. gradient_descent(X, y, weights, learning_rate, iterations)
Purpose: To perform gradient descent optimization to minimize the cost function and update the weights.

Explanation:
Gradient descent is an iterative optimization algorithm used to find the minimum of a function. In the context of logistic regression, it is used to minimize the cost function by updating the weights in the direction of the steepest descent (negative gradient).

Steps:

Compute the gradient of the cost function with respect to each weight.
Update the weights by subtracting the product of the learning rate and the gradient.
Mathematical Formulation:

𝑤
:
=
𝑤
−
𝛼
1
𝑚
∑
𝑖
=
1
𝑚
(
ℎ
𝑤
(
𝑥
(
𝑖
)
)
−
𝑦
(
𝑖
)
)
𝑥
(
𝑖
)
w:=w−α
m
1
​
  
i=1
∑
m
​
 (h
w
​
 (x
(i)
 )−y
(i)
 )x
(i)

where
𝛼
α is the learning rate.

In [ ]:
def gradient_descent(X, y, weights, learning_rate, iterations):
    m = len(y)
    cost_history = np.zeros(iterations)

    for i in range(iterations):
        weights -= learning_rate * (1/m) * np.dot(X.T, (predict(X, weights) - y))
        cost_history[i] = compute_cost(X, y, weights)

    return weights, cost_history


**Usage**:

X is the matrix of input features.

y is the vector of true labels.

weights is the vector of model parameters.

learning_rate is a hyperparameter that controls the step size in each iteration.

iterations is the number of iterations to perform.

Returns the optimized weights and the history of the cost function over iterations.

#Train the Model

In [ ]:
# Set hyperparameters
learning_rate = 0.01
iterations = 1000
# Train the model
weights, cost_history = gradient_descent(X_train, y_train, weights, learning_rate, iterations)


<ipython-input-13-dc64dee5606b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-41-79fb628300cc>:4: RuntimeWarning: divide by zero encountered in log
  cost = -1/m * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))
<ipython-input-41-79fb628300cc>:4: RuntimeWarning: invalid value encountered in multiply
  cost = -1/m * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))


#Make Predictions

In [ ]:
# Make predictions on the test set
y_pred_prob = predict(X_test, weights)
y_pred = (y_pred_prob >= 0.5).astype(int)

<ipython-input-13-dc64dee5606b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


#Evaluate the Model

In [ ]:
# Calculate accuracy
accuracy = np.mean(y_pred == y_test) * 100

print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 97.00%


#SIMPLE Weather Report

In [ ]:
temperature_now= float(input("Enter the present temperature ranges between (0 to 60)*c:\n"))
humidity_now= float(input("Enter the present humidity ranges between (0 to 100)%:\n"))
pressure_now= float(input("Enter the present pressure ranges between (0 to 3000)hpa:\n"))

# Combine into a single dataset
X_test_now = np.column_stack((humidity_now,temperature_now,pressure_now))
# Add a column of ones to X for the bias term
X_test_now = np.hstack((np.ones((X_test_now.shape[0], 1)), X_test_now))
y_pred_prob_now = predict(X_test_now, weights)

print('the rainfall probabilty', y_pred_prob_now*100)
print('the sunny probabilty', 100-y_pred_prob_now*100)

Enter the present temperature ranges between (0 to 60)*c:
25
Enter the present humidity ranges between (0 to 100)%:
30
Enter the present pressure ranges between (0 to 3000)hpa:
1000
the rainfall probabilty [[0.]]
the sunny probabilty [[100.]]


<ipython-input-13-dc64dee5606b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


#YOUR TASK IS TO MAKE THIS **SIMPLE WEATHER REPORT** EVEN MORE **PRECISE** with your OWN Technics.  

---
**Tag
your Github reposite , who ever finished Task**